In [1]:
# Preprocessing.ipynb

# 1. Introduzione e Setup
# Questo notebook esegue il preprocessing dei file audio, uniformandoli ad una lunghezza predefinita.

# Importazione delle librerie necessarie
import os
import numpy as np
import librosa
import soundfile as sf

# 2. Definizione dei Parametri e delle Directory
# Definizione delle directory di input e output
input_dir = 'Dataset'
output_dir = 'NuovoDataset'

# Creare la directory di output se non esiste
os.makedirs(output_dir, exist_ok=True)

# Definizione della lunghezza desiderata in secondi e in campioni
desired_length_sec = 4  # Durata desiderata in secondi
sampling_rate = 192000  # Frequenza di campionamento (modifica se necessario)
desired_length_samples = int(desired_length_sec * sampling_rate)

# 3. Raccolta dei File Audio
# Lista per memorizzare i percorsi dei file audio
audio_files = []

# Scorrere ricorsivamente le cartelle nel dataset
for root, dirs, files in os.walk(input_dir):
    for file in files:
        if file.endswith('.wav'):
            audio_files.append(os.path.join(root, file))

# Visualizzare il numero di file audio trovati
print(f"Numero di file audio trovati: {len(audio_files)}")

# 4. Funzione di Preprocessing
# Funzione per tagliare e salvare l'audio
def preprocess_audio(file, output_dir, desired_length_samples, sampling_rate):
    y, sr = librosa.load(file, sr=sampling_rate, mono=False)  # Carica come stereo
    if y.ndim == 1:  # Se è mono, aggiungi una dimensione
        y = y[np.newaxis, :]
    
    y_trimmed = librosa.util.fix_length(y, size=desired_length_samples, axis=-1)
    
    # Creare la struttura delle directory di output mantenendo la stessa gerarchia
    relative_path = os.path.relpath(file, input_dir)
    output_file = os.path.join(output_dir, relative_path)
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    
    sf.write(output_file, y_trimmed.T, sr)  # Trasponi per ottenere (samples, channels)
    print(f"Processed {file} -> {output_file}")

# 5. Applicazione della Funzione di Preprocessing
# Preprocessamento di ciascun file audio
for file in audio_files:
    preprocess_audio(file, output_dir, desired_length_samples, sampling_rate)

print("Tutti i file audio sono stati tagliati alla lunghezza desiderata e salvati mantenendo la struttura delle categorie.")

Numero di file audio trovati: 2596
Processed Dataset/Target/Acoustic Harassment Devices/AHD_Airmar.wav -> NuovoDataset/Target/Acoustic Harassment Devices/AHD_Airmar.wav
Processed Dataset/Target/Tug/20171128c-28_190038.wav -> NuovoDataset/Target/Tug/20171128c-28_190038.wav
Processed Dataset/Target/Tug/20171227a-66_110853.wav -> NuovoDataset/Target/Tug/20171227a-66_110853.wav
Processed Dataset/Target/Tug/20171127a-24_105442.wav -> NuovoDataset/Target/Tug/20171127a-24_105442.wav
Processed Dataset/Target/Tug/20171210-49_49.wav -> NuovoDataset/Target/Tug/20171210-49_49.wav
Processed Dataset/Target/Tug/20171104-1_200959.wav -> NuovoDataset/Target/Tug/20171104-1_200959.wav
Processed Dataset/Target/Tug/20171128-25_052238.wav -> NuovoDataset/Target/Tug/20171128-25_052238.wav
Processed Dataset/Target/Tug/20171205-44_145246.wav -> NuovoDataset/Target/Tug/20171205-44_145246.wav
Processed Dataset/Target/Tug/20171221-61_114923.wav -> NuovoDataset/Target/Tug/20171221-61_114923.wav
Processed Dataset/T